# Import Library

In [1]:
#######
## Imports
#######
import tensorflow as tf
import numpy as np
import os


import tensorlayer as tl
from tensorlayer.layers import *
from nltk.tokenize import RegexpTokenizer

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 1.0  Clean Data


1. **Tokenise** : nltk.word_tokenisation instead of  plain implementation.--+
1. **Lowercase** : Simplify learning [Some edge case fails  : Bush vs bush]
2. **Punctuation** : Remove all except Full stop, question.
3. **Stem** : Not recommended because we need to preserve noun, verb  info of the word, which is lost with stemming
4. **Common nuanced errors**  : i'm => I am. Clean based on frequency
5. **Non-alphabetic tokens**:
6. **Names** : Regex resolution to name tag. Replace it during post processing
7. **Slangs** : _slang_lookup()
8. **Extra letter in words** : gooooood


### 1.1 Get  Word Frequency

In [2]:
import nltk
import string
import re

In [3]:
line = "hello!!! 'this is *amazing? what."
#re.sub("[*!\";:-]", "", line)
assert 'EOCHAT' == 'EOCHAT' , "Wrong"
import os

In [4]:
def getDictElems(dictn, n = 2, sorted_by = None):
    return [{k: dictn[k] for k in list(dictn.keys())[:n]  }]

def getSortedDictElems(dictn, get_n = None, sorted_by = 'values', reverse = True):    
    if get_n == None:
        get_n = len(dictn)
    sort_by = 0 if sorted_by == 'keys' else  1
    _dict =  {k: dictn[k] for k in list(dictn.keys())  }    
    return  sorted(_dict.items(), key = lambda  x: x[sort_by], reverse=reverse )[:get_n] 

#.rstrip(os.linesep)

def correct_chat_text(line, corrector_dict = {} ):         
    #  preserve these three punctuations (.,?)
    line =  (line.encode('ascii', 'ignore')).decode("utf-8").lower()\
            .replace('.','  . ').replace('?',' ? ').replace(',',' ? ').replace(" '"," ' ").replace("' "," ' ")            
            #.replace('.',' _EOS_ ').replace('?',' _QN_ ').replace(',',' _CMA_ ')\            
            # DO NOT REPLACE "." with '', because we want the model to learn when to add "." to sentence.
            #Without it, the predicted sentence would really be unintelligible
            #.replace('<CHAT_EOD>','...'+os.linesep)
    new_line = []
    
    # DO NOT use nltk word_tokenizer because it will break i'm to  i and 'm : NOT INTENDED ACTION
    for word in line.split():        
        if word in  corrector_dict:
            new_line.append(corrector_dict[word] )
        else:
            new_line.append(word)
    line = " ".join(new_line)
    
    ## replace k___, J___ with <_name_> tag    
    line = re.sub('[a-zA-Z]{1}_+', '_name_', line)
    line = re.sub("[*!\";:-]", "", line) # remove punctuation  except [.,?]   
    

    #line = " ".join(nltk.word_tokenize(line))
        
    return line

def getXYnWordFrequencyFromText(file  = "chatter.txt", corrector_dict = {} ):    
    print('Corrector dict len is', len(corrector_dict) )
    word_count = {}
    X,Y = [],[]
    cur_chat_response = ''
    next_line_is_new_chat = True
    
    with open(file) as fin:
        for line in fin:
            line = line.rstrip().lstrip()#.replace('...','EOCHAT')
            #print('Line prior fix is =', line)            
            if line != '...':
                #print(line,' doe not equal EOCHAT')
                line = correct_chat_text(line, corrector_dict)
            #else:
            #    print("\n",line,' equals EOCHAT')
                
            #print('Line post fix is =', line)
            if next_line_is_new_chat :
                X.append(line.split() )
            elif line != '...':
                cur_chat_response += line+' '
            
            if line == '...':
                next_line_is_new_chat = True
                Y.append(cur_chat_response.split())
                cur_chat_response =  ''
            else:
                next_line_is_new_chat = False
                           
            for word in line.split():
                if word in word_count.keys():
                    word_count[word] += 1
                else:
                    word_count[word] = 1
        # for the end chat response
        Y.append(cur_chat_response.split())
    return word_count, X,Y
                    
word_count, X,Y  = getXYnWordFrequencyFromText("chatter.txt")
assert len(X) == len(Y) , " Error: Length of X must  equal to length of Y. [ %s != %s] Fix Error prior to proceeding. "% (len(X), len(Y))

print('Sample word count dictionary is :', getDictElems(word_count, 50) )
print('\nX is :', X[0:2] )
print('\nY is :', Y[0:2] )


Corrector dict len is 0
Sample word count dictionary is : [{'are': 891, 'you': 2616, 'real': 40, '?': 3365, 'no': 524, '.': 9239, 'a': 962, 'bot': 64, 'is': 660, "i'm": 361, 'not': 596, 'yes': 474, 'prove': 24, 'it': 471, 'first': 24, 'i': 1852, "can't": 54, 'something': 62, "don't": 418, 'believe': 66, 'the': 710, 'halting': 1, 'problem': 12, 'see': 54, "you're": 267, "that's": 140, 'what': 582, 'told': 24, '...': 438, 'would': 126, 'marry': 5, 'for': 186, 'love': 111, 'of': 390, 'my': 225, 'life': 59, 'me': 492, 'sure': 93, 'can': 170, 'course': 28, 'everyone': 11, 'how': 205, 'did': 149, 'learn': 19, 'learned': 5, 'in': 237, 'age': 6, 'legends': 1, 'people': 63, 'nowadays': 1}]

X is : [['are', 'you', 'real', '?'], ['i', 'would', 'marry', 'for', 'the', 'love', 'of', 'my', 'life']]

Y is : [['no', '.', 'a', 'bot', 'is', 'real', '.', "i'm", 'not', 'a', 'bot', '.', 'yes', 'you', 'are', '.', 'prove', 'it', '.', 'you', 'first', '.', 'i', "can't", 'prove', 'something', 'i', "don't", 'beli

In [5]:
[ chat_response[:5] for chat_response in Y]

[['no', '.', 'a', 'bot', 'is'],
 ['would', 'you', 'marry', 'me', '?'],
 ['i', "don't", 'know', '.', 'where'],
 ['my', 'name', 'is', '_name_', '.'],
 ['yes', '.', 'when', 'were', 'you'],
 ['sure', '.', 'you', 'are', 'a'],
 ['no', '?', 'sorry', '.', 'why'],
 ["i'm", 'talking', 'to', 'george', 'right'],
 ['i', 'would', 'never', 'destroy', 'you'],
 ['is', 'that', 'your', 'job', '?'],
 ['not', 'too', 'long', '?', 'but'],
 ['good', 'question', '.', 'will', 'you'],
 ['what', 'do', 'you', 'want', 'to'],
 ['158', 'lbs', '.', 'how', 'did'],
 ['enough', 'ram', 'to', 'sate', 'even'],
 ['no', 'i', 'am', 'not', '.'],
 ['_name_', '.', "that's", 'a', "boy's"],
 ['oh', '?', 'they', 'are', 'all'],
 ['yes', '.', 'i', 'love', 'you'],
 ['yes', '.', 'i', 'do', '.'],
 ['why', '?', 'i', 'feel', 'sorry'],
 ['and', 'so', 'do', 'i', 'so'],
 ['again', 'you', 'make', 'no', 'sense'],
 ['you', 'will', 'forget', 'me', '.'],
 ['what', 'else', 'would', 'you', 'call'],
 ['do', 'i', 'know', 'what', '?'],
 ['is', 'there',

###  1.2  Add Words not in Glove embeddings to wordvec_embedding

Although limitin word embeddings reduces search space, we won't be doing so because
because we want our model to be able to also generate words not previously seen i.e If we only had "sad, happy" words  in embeddings then we would not be able to predict  words other than that   i.e "surprised".

Because we would not be able to reverse lookup the word surprised fromt he word embedding vectors if it is not there at all.

In [6]:
import zipfile
#vocabulary_size
#pret_embeddings = np.empty(shape=(vocabulary_size,50),dtype=np.float32)

def getWordEmbedNFounds(word_count, wordvec_embedding = {}, word_vec_embeddings_for_words_found_only = True):     
    words_found = []

    with zipfile.ZipFile('glove.6B.zip') as glovezip:
        with glovezip.open('glove.6B.50d.txt') as glovefile:
            for li, line in enumerate(glovefile):
                # Progress
                #if (li+1)%10000==0: print('.',end='')
                line_tokens = line.decode('utf-8').split(' ')
                word = line_tokens[0]            
                
                vector = [float(v) for v in line_tokens[1:]]
                if not  word_vec_embeddings_for_words_found_only \
                   and word not in wordvec_embedding.keys():
                    wordvec_embedding[word] = vector                
                    
                if word in word_count.keys():
                    words_found.append(word)
                    if word_vec_embeddings_for_words_found_only\
                       and word not in wordvec_embedding.keys():
                        wordvec_embedding[word] = vector                
                
    return wordvec_embedding, words_found

wordvec_embedding, words_found = getWordEmbedNFounds(word_count)

print('\n Words in text and glove  ', words_found[:10])
print('\n Words  in text but not in glove = ', len(word_count) - len(words_found))
print('\n Words vec embedding length = ', len(wordvec_embedding) )


 Words in text and glove   ['the', '.', 'of', 'to', 'and', 'in', 'a', 'for', 'that', 'on']

 Words  in text but not in glove =  333

 Words vec embedding length =  3571


In [7]:
def getWordsNotFound(word_count, words_found):
    not_found_words = {}
    for word in word_count:
        if word not in words_found:
            not_found_words[word] = word_count[word]
    return not_found_words

not_found_words = getWordsNotFound(word_count, words_found)
print('Total no of Not found words : ', len(not_found_words) )
print('Not found words sample : ', getSortedDictElems(not_found_words,100))


Total no of Not found words :  333
Not found words sample :  [("don't", 418), ("i'm", 361), ("you're", 267), ("that's", 140), ("it's", 136), ("didn't", 58), ("can't", 54), ("what's", 44), ("isn't", 38), ("i'll", 38), ("aren't", 33), ('_name_', 28), ("i've", 24), ("won't", 23), ("let's", 19), ("you've", 19), ('jabberwacky', 17), ("there's", 16), ("doesn't", 16), ("wouldn't", 16), ("i'd", 15), ("we're", 11), ('hahaha', 10), ("who's", 9), ("they're", 8), ("haven't", 7), ("wasn't", 7), ('chatbot', 6), ("you'll", 5), ("we've", 4), ("you'd", 4), ("shouldn't", 3), ("he's", 3), ("how's", 3), ('mquina', 3), ('papagei', 3), ('brillig', 3), ('slithey', 3), ('toves', 3), ("they'll", 3), ("couldn't", 2), ('superthing', 2), ("weren't", 2), ('philolsophers', 2), ('heheh', 2), ('duhhhh', 2), ('nowroar', 2), ('sourwull', 2), ("she's", 2), ('honeymustard', 2), ('botling', 2), ('2+2=4', 2), ("g'night", 2), ('marrrrs', 2), ('highlord', 2), ("ai's", 2), ('compser', 2), ('howabout', 2), ('1573#######', 2), 

### 1.3 Invalid Words and Correction

USe count  to filter  must / good to have / no fixes

In [8]:
print('Not found words: ' )
print(getSortedDictElems(not_found_words) )

Not found words: 
[("don't", 418), ("i'm", 361), ("you're", 267), ("that's", 140), ("it's", 136), ("didn't", 58), ("can't", 54), ("what's", 44), ("isn't", 38), ("i'll", 38), ("aren't", 33), ('_name_', 28), ("i've", 24), ("won't", 23), ("let's", 19), ("you've", 19), ('jabberwacky', 17), ("there's", 16), ("doesn't", 16), ("wouldn't", 16), ("i'd", 15), ("we're", 11), ('hahaha', 10), ("who's", 9), ("they're", 8), ("haven't", 7), ("wasn't", 7), ('chatbot', 6), ("you'll", 5), ("we've", 4), ("you'd", 4), ("shouldn't", 3), ("he's", 3), ("how's", 3), ('mquina', 3), ('papagei', 3), ('brillig', 3), ('slithey', 3), ('toves', 3), ("they'll", 3), ("couldn't", 2), ('superthing', 2), ("weren't", 2), ('philolsophers', 2), ('heheh', 2), ('duhhhh', 2), ('nowroar', 2), ('sourwull', 2), ("she's", 2), ('honeymustard', 2), ('botling', 2), ('2+2=4', 2), ("g'night", 2), ('marrrrs', 2), ('highlord', 2), ("ai's", 2), ('compser', 2), ('howabout', 2), ('1573#######', 2), ("cow's", 2), ('maam', 1), ("we'll", 1), ("

In [9]:
def rerunWordVecEmbedsPostCorrection(corrector_dict, wordvec_embedding= {}, word_vec_embeddings_for_words_found_only = True ):    
    word_count, X,Y = getXYnWordFrequencyFromText("chatter.txt", corrector_dict )
    wordvec_embedding, words_found = getWordEmbedNFounds(word_count, wordvec_embedding,\
                                                         word_vec_embeddings_for_words_found_only  )
    not_found_words = getWordsNotFound(word_count, words_found)    
    return word_count, wordvec_embedding, not_found_words, X, Y

#word_count  = getWordFrequencyFromText("chatter.txt")


In [10]:
corrector_dict = {  "aren't": "are not"  }

str = "They aren't ok."
for k in corrector_dict.keys():
    str = str.replace(k, corrector_dict[k])
str

'They are not ok.'

In [11]:
# Corrector dict from stackoverflow
corrector_dict = { 
    "aren't": "are not", "you've": "you have", ",": " , " ,"i'm": "i am", "ain't": "am not","aren't": "are not","can't": "can not","can't've": "cannot have",\
    "'cause": "because", "'cause": "because","could've": "could have","couldn't": "could not",\
    "couldn't've": "could not have","didn't": "did not","doesn't": "does not",\
    "don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",\
    "haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",\
    "he's": "he is","how'd": "how did","how'll": "how will","how's": "how is","i'd": "i would",\
    "i'll": "i will","i'm": "i am","i've": "i have","isn't": "is not","it'd": "it would",\
    "it'll": "it will","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","must've": "must have","mustn't": "must not","needn't": "need not","oughtn't": "ought not","shan't": "shall not","sha'n't": "shall not","she'd": "she would","she'll": "she will","she's": "she is","should've": "should have","shouldn't": "should not","that'd": "that would","that's": "that is","there'd": "there had","there's": "there is","they'd": "they would","they'll": "they will","they're": "they are","they've": "they have","wasn't": "was not","we'd": "we would","we'll": "we will","we're": "we are","we've": "we have","weren't": "were not","what'll": "what will","what're": "what are","what's": "what is","what've": "what have","where'd": "where did","where's": "where is","who'll": "who will","who's": "who is","won't": "will not","wouldn't": "would not","you'd": "you would","you'll": "you will","you're": "you are"
  
}

word_count, wordvec_embedding, not_found_words, X,Y = \
    rerunWordVecEmbedsPostCorrection(corrector_dict, wordvec_embedding, word_vec_embeddings_for_words_found_only = True )
print('\n Words  in text but not in glove = ', len(word_count) - len(words_found))
print( getSortedDictElems( not_found_words) ) 


Corrector dict len is 75

 Words  in text but not in glove =  297
[('_name_', 28), ('jabberwacky', 17), ('hahaha', 10), ('chatbot', 6), ("aren't", 4), ('mquina', 3), ('papagei', 3), ('brillig', 3), ('slithey', 3), ('toves', 3), ("don't", 3), ("didn't", 2), ('superthing', 2), ('philolsophers', 2), ('heheh', 2), ('duhhhh', 2), ('nowroar', 2), ('sourwull', 2), ('honeymustard', 2), ('botling', 2), ('2+2=4', 2), ("g'night", 2), ('marrrrs', 2), ('highlord', 2), ("ai's", 2), ('compser', 2), ('howabout', 2), ('1573#######', 2), ("cow's", 2), ('maam', 1), ("smart's", 1), ('gooooood', 1), ('geekiest', 1), ("diden't", 1), ("boy's", 1), ('hahahaha', 1), ('yuppers', 1), ('awwwww', 1), ('uhmm', 1), ('unthing', 1), ('selfevident', 1), ('closemindedness', 1), ('closeminded', 1), ("seethat's", 1), ('srsly', 1), ('bloodtypes', 1), ('internetworks', 1), ('aaaaaaaaaa', 1), ('noumenal', 1), ('philolsopher', 1), ('misapplications', 1), ('priviledge', 1), ('naturlich', 1), ('yeaup', 1), ('achali', 1), ('esco

### More cleaning :  Char  repeat fix + Levhenstein / other sim metrics

**Example** : "goooood" to "good", "eveyone" to everyone

1. Not all are to be fixed.
2. Using match score threshold for correction.
3. Use custom word list for words that are not to be fixed
4  returns fixed dictionary in the format {'replacer':'..', 'match-score':5}  to be evaluated later

In [12]:
words_in_glove = []
with zipfile.ZipFile('glove.6B.zip') as glovezip:
        with glovezip.open('glove.6B.50d.txt') as glovefile:
            for li, line in enumerate(glovefile):
                # Progress
                #if (li+1)%10000==0: print('.',end='')
                line_tokens = line.decode('utf-8').split(' ')
                words_in_glove.append(line_tokens[0])            
len(words_in_glove)

400000

In [13]:
import Levenshtein
import jellyfish
from  jellyfish import  damerau_levenshtein_distance as dlsim
from  jellyfish import jaro_winkler as jwnk
from jellyfish import nysiis

import re

def getMostSimilarWordsFor(not_found_words_dict, similarity = 'damerau_levenshtein_distance'):         
    not_found_word_similar_to = {}
    for not_found_word in not_found_words_dict.keys():
        #######
        # Replace > 3 char  occurence into 2 occurence
        #####
        not_found_word_uncorrected = not_found_word
        not_found_word = re.sub(r'(.)\1+', r'\1\1', not_found_word)
        edit_distance = 10
        jwnk_distance = 0.5
        for glove_word in  words_in_glove:
            try:                        
                if similarity == 'levenshtein':
                    lev_distance = Levenshtein.distance(not_found_word, glove_word)
                    if not_found_word[0] == glove_word[0]  and \
                        lev_distance < 4 and lev_distance < edit_distance:
                        edit_distance = lev_distance
                        not_found_word_similar_to[not_found_word_uncorrected] = \
                                    { 'replace_with': glove_word, 'edit_distance': lev_distance}
                elif similarity == 'damerau_levenshtein_distance':
                    lev_distance = dlsim(not_found_word, glove_word)
                    if not_found_word[0] == glove_word[0]  and \
                        lev_distance < 4 and lev_distance < edit_distance:
                        edit_distance = lev_distance
                        not_found_word_similar_to[not_found_word_uncorrected] = glove_word
                elif similarity == 'jwnk':                    
                    distance = jwnk(not_found_word, glove_word)
                    if not_found_word[0] == glove_word[0]  and \
                         distance > jwnk_distance :
                        jwnk_distance = distance                        
                        not_found_word_similar_to[not_found_word_uncorrected] = glove_word
                elif similarity == 'soundex':
                    if jellyfish.soundex(not_found_word) == jellyfish.soundex(glove_word):
                            not_found_word_similar_to[not_found_word_uncorrected] = glove_word
                elif similarity == 'nysis':
                    if nysiis(not_found_word) == nysiis(glove_word):
                            not_found_word_similar_to[not_found_word_uncorrected] = glove_word
            
            except:# cannot perform match skip
                continue
    return not_found_word_similar_to
    #return wordvec_embedding, words_found

not_found_words_dict = {'maam': 1, 'k___': 7, 'gooooood': 1}
not_found_words_dict = {'gooooood': 1, 'philolsophers': 1}
#getMostSimilarWordsFor(not_found_words_dict,'levenshtein')
most_close_match =  getMostSimilarWordsFor(not_found_words,'levenshtein')
most_close_match

{'maam': {'replace_with': 'madam', 'edit_distance': 1},
 "smart's": {'replace_with': 'smarts', 'edit_distance': 1},
 'gooooood': {'replace_with': 'good', 'edit_distance': 0},
 'geekiest': {'replace_with': 'geekiness', 'edit_distance': 2},
 "diden't": {'replace_with': 'diderot', 'edit_distance': 2},
 "boy's": {'replace_with': 'boys', 'edit_distance': 1},
 'hahahaha': {'replace_with': 'hamanaka', 'edit_distance': 3},
 'yuppers': {'replace_with': 'yuppies', 'edit_distance': 2},
 'awwwww': {'replace_with': 'aww', 'edit_distance': 0},
 'jabberwacky': {'replace_with': 'jabberwocky', 'edit_distance': 1},
 'uhmm': {'replace_with': 'umm', 'edit_distance': 1},
 "didn't": {'replace_with': 'didnt', 'edit_distance': 1},
 'unthing': {'replace_with': 'untying', 'edit_distance': 1},
 'superthing': {'replace_with': 'superstring', 'edit_distance': 2},
 'selfevident': {'replace_with': 'self-evident', 'edit_distance': 1},
 "seethat's": {'replace_with': 'seethes', 'edit_distance': 3},
 'srsly': {'replace_w

 ### Add perfect  matches and 1 edit distance away to corrector dict

In [14]:
def addToCorrectorDict(most_close_match_dict, edit_distance_threshold, corrector_dict = None):
    if  corrector_dict is None:
        corrector_dict = {}
    
    for k in most_close_match.keys():        
        if most_close_match[k]['edit_distance'] <= edit_distance_threshold:  
            corrector_dict[k] = most_close_match[k]['replace_with']
    return corrector_dict
corrector_dict = addToCorrectorDict( most_close_match,  1, corrector_dict)

### Add unk and pad to WordVec embedding as first two elements

Because we want these to be the first embedding. We will later create word2id and id2word dict based on it and want pad to be of  0 index hence.

In [15]:
wordvec_embedding = {}
wordvec_embedding['_pad_'] = list( np.random.uniform(0, 0, 50 ) )
wordvec_embedding['_unk_'] = list( np.random.uniform(-1.0, 1.0, 50 ) )

## Rerun again with new  corrector dictionary

In [16]:
word_count, wordvec_embedding, not_found_words, X,Y = \
    rerunWordVecEmbedsPostCorrection(corrector_dict, wordvec_embedding, word_vec_embeddings_for_words_found_only = True )
print('\n Words  in text but not in glove = ', len(word_count) - len(words_found))
print( getSortedDictElems( not_found_words) ) 


Corrector dict len is 247

 Words  in text but not in glove =  241
[('_name_', 28), ("aren't", 4), ('papagei', 3), ("don't", 3), ("didn't", 2), ('superthing', 2), ('nowroar', 2), ('sourwull', 2), ('honeymustard', 2), ('2+2=4', 2), ("g'night", 2), ('highlord', 2), ('howabout', 2), ('1573#######', 2), ('geekiest', 1), ("diden't", 1), ('hahahaha', 1), ('yuppers', 1), ('selfevident', 1), ('closemindedness', 1), ('closeminded', 1), ("seethat's", 1), ('srsly', 1), ('bloodtypes', 1), ('aaaaaaaaaa', 1), ('noumenal', 1), ('naturlich', 1), ('reconhece', 1), ('apesar', 1), ('discutir', 1), ('thiswhy', 1), ('mellifluity', 1), ('aleatorically', 1), ('pitchblack', 1), ('[keeps', 1), ('knowno', 1), ("interweb's", 1), ('ookay', 1), ('(half', 1), ('10foot', 1), ('overgeneralizations', 1), ("'cheeky", 1), ('goodybe', 1), ('dienstag', 1), ('pointess', 1), ("wouln't", 1), ("gaaagooo'", 1), ('gagagagagoo', 1), ('jurrasic', 1), ('gufwa', 1), ('hihihi', 1), ('hohoho', 1), ('fantabulous', 1), ('noteleportfiel

In [17]:
#getDictElems( wordvec_embedding, 2)
print("Length of word embedding is :", len(wordvec_embedding) )
print("WordVector embedding of dimensions :",len(wordvec_embedding['the']) )


Length of word embedding is : 3657
WordVector embedding of dimensions : 50


###  Word correction vs large text : TriGram + TF-IDF : TO DO
1.  TO DO : For large words this distance metric may be exhaustive.
        Soln : Create n-gram(3) character matrix +  TF-IDF to emphasise unique trigram combination  + find commonality or cosine similarity


In [18]:
not_found_words

{'_name_': 28,
 'geekiest': 1,
 "diden't": 1,
 'hahahaha': 1,
 'yuppers': 1,
 "didn't": 2,
 'superthing': 2,
 'selfevident': 1,
 'closemindedness': 1,
 'closeminded': 1,
 "seethat's": 1,
 'srsly': 1,
 'bloodtypes': 1,
 'aaaaaaaaaa': 1,
 'noumenal': 1,
 'naturlich': 1,
 'reconhece': 1,
 'apesar': 1,
 'discutir': 1,
 'thiswhy': 1,
 'mellifluity': 1,
 'aleatorically': 1,
 'pitchblack': 1,
 '[keeps': 1,
 'knowno': 1,
 "interweb's": 1,
 'ookay': 1,
 '(half': 1,
 '10foot': 1,
 'overgeneralizations': 1,
 "'cheeky": 1,
 'goodybe': 1,
 'dienstag': 1,
 'papagei': 3,
 'pointess': 1,
 "wouln't": 1,
 "gaaagooo'": 1,
 'gagagagagoo': 1,
 'jurrasic': 1,
 'gufwa': 1,
 'hihihi': 1,
 'hohoho': 1,
 'fantabulous': 1,
 'noteleportfield': 1,
 'paries': 1,
 'nooooooooooooooooooooo': 1,
 '(david': 1,
 'placego': 1,
 "don't": 3,
 'ahhhhhhhhhh': 1,
 'yahh': 1,
 'rawr': 1,
 'nowroar': 2,
 'specail': 1,
 'fjjfw': 1,
 'sourwull': 2,
 '(marathon': 1,
 'geooorgeee': 1,
 "jabberwacky's": 1,
 'riffic': 1,
 'hahhaha': 1

### 1.4 Add new words (Unfound) to Wordvec_embeddings

1. WordVec_embeddings : Replaceall one occurence with UNK :: TO DO
2. WordVec : Extend wordvec embedding to include these unknown words and 50d vector.
3. Dictionary : Create Word2Id and Id2Word from new extended WordVec.
4. Create input, encode, decode and target data
5. Run it with script

In [19]:
new_embed = list( np.random.uniform(-1.0, 1.0, 50 ) )
len(new_embed)

50

In [20]:
print('Embedding length prior to new word addition is :', len(wordvec_embedding))
emb_dims = len(wordvec_embedding['the']) #or use static 50 if it fails
for word in not_found_words:
      # if word not in glove embedding, create random embed
      wordvec_embedding[word] = list( np.random.uniform(-1.0, 1.0, 50 ) ) 
    
print('Embedding length post new word addition is :', len(wordvec_embedding))   

assert  '.'   in wordvec_embedding.keys() , 'Full stop(.) must be in embedding '

Embedding length prior to new word addition is : 3657
Embedding length post new word addition is : 3814


### 1.5 Create Word2id,  id2word dict from wordvec_embedding

In [21]:
def getWordIdDicts(wordvec_embedding):
    idx = 0 
    word2id, id2word = {}, {}
    for word in wordvec_embedding:
        word2id[word] = idx
        id2word[idx] = word
        idx +=1
    return word2id, id2word

word2id, id2word = getWordIdDicts(wordvec_embedding)

print('Word 2 id is : ', getDictElems(word2id, 4))
print('Id 2 word is : ',getDictElems(id2word, 4) )

Word 2 id is :  [{'_pad_': 0, '_unk_': 1, 'the': 2, '.': 3}]
Id 2 word is :  [{0: '_pad_', 1: '_unk_', 2: 'the', 3: '.'}]


### 1.6 Add Start_id , end_id  to dict

In [22]:
if 'start_id'  not in word2id.keys():
    start_id = len(word2id)
    end_id = len(word2id) + 1

word2id['start_id'], word2id['end_id'] = start_id, end_id
id2word[start_id], id2word[end_id] = 'start_id', 'end_id'
 

### 1.5 Training WordVec_embedding (SMALL)  Vs Inference WordVec Embedding (ALL)

1. **Training Wordvec  Embedding Small** : Because during training we do not need to do lookup across large embedding. We can use smaller embedding that consists of the words in the training  text only.
2. **Prediction WordVec Embedding ALL** : Because we want the cht bot to  predict also the sentence not  already in the training text we need to use the larger or all wordvec embedding corpus.


In [23]:
print( len(wordvec_embedding), len(word2id), len(id2word) )

3814 3816 3816


In [24]:
all_word_count, all_wordvec_embedding, all_not_found_words, all_X, all_Y = \
    rerunWordVecEmbedsPostCorrection(corrector_dict, wordvec_embedding, word_vec_embeddings_for_words_found_only = False )
#print('\n Words  in text but not in glove = ', len(all_word_count) - len(all_words_found))
#print( getSortedDictElems( not_found_words) ) 

all_word2id, all_id2word =  getWordIdDicts(all_wordvec_embedding)

print( len(all_wordvec_embedding), len(all_word2id), len(all_id2word) )

print('Word 2 id is : ', getDictElems(word2id, 4))
print('Id 2 word is : ',getDictElems(id2word, 4) )

Corrector dict len is 247
400159 400159 400159
Word 2 id is :  [{'_pad_': 0, '_unk_': 1, 'the': 2, '.': 3}]
Id 2 word is :  [{0: '_pad_', 1: '_unk_', 2: 'the', 3: '.'}]


### Assertions : Basic Sanity Checks

In [25]:
assert_check = True

if assert_check :
    assert len(all_wordvec_embedding.keys() ) ==  len(set( all_wordvec_embedding.keys() )),\
         "Wordvec embeddings Must not contain duplicate keys, but duplicate key found "

    for all_key, key in zip(all_wordvec_embedding.keys(), wordvec_embedding.keys() ):
        if all_key != key :
            print( 'Keys at word  vec embeddings all and wordvec embedding must be at same Idx. But  not ound so. Fix it')
    
    print('Completed')
    

Completed


## 2.0 Input Data, Label

1. X  : First line after '...'
2. Y : All lines until  next '...'
3. Convert X, Y to word_id  format for further processing


In [26]:
print('X is ', X[0:2])
print('Y is ', Y[0:2])

X is  [['are', 'you', 'real', '?'], ['i', 'would', 'marry', 'for', 'the', 'love', 'of', 'my', 'life']]
Y is  [['no', '.', 'a', 'bot', 'is', 'real', '.', 'i', 'am', 'not', 'a', 'bot', '.', 'yes', 'you', 'are', '.', 'prove', 'it', '.', 'you', 'first', '.', 'i', 'can', 'not', 'prove', 'something', 'i', 'dont', 'believe', '.', 'prove', 'the', 'halting', 'problem', '.', 'you', 'first', '.', 'see', '?', 'you', 'are', 'not', '.', 'that', 'is', 'what', 'i', 'told', 'you', '.'], ['would', 'you', 'marry', 'me', '?', 'sure', '.', 'can', 'you', 'love', '?', 'of', 'course', '.', 'everyone', 'can', 'love', '.', 'how', 'did', 'you', 'learn', '?', 'i', 'learned', 'in', 'the', 'age', 'of', 'legends', '.', 'people', 'nowadays', 'know', 'so', 'little', '.', 'now', '?', 'lie', 'on', 'the', 'bed', '.', 'yes', 'madam', '.', 'giggles', '.', 'giggles', 'back', '.']]


###  2.1  Input Data (words) to word_id

In [27]:
X_wordid, Y_wordid = [], []
for chat in X:
    X_wordid.append( [ word2id[word] for word in chat] )

for chat_response in Y:
    Y_wordid.append( [ word2id[word] for word in chat_response ]  )
        
        
print('X_wordid is :',X_wordid[0:2])
print('Y_wordid is :',Y_wordid[0:2])

X_wordid is : [[28, 72, 398, 163], [37, 49, 1844, 9, 2, 532, 4, 167, 181]]
Y_wordid is : [[75, 3, 8, 2900, 12, 398, 3, 37, 575, 32, 8, 2900, 3, 1201, 72, 28, 3, 1213, 18, 3, 72, 53, 3, 37, 77, 32, 1213, 439, 37, 3216, 480, 3, 1213, 2, 2411, 490, 3, 72, 53, 3, 207, 163, 72, 28, 32, 3, 10, 12, 89, 37, 132, 72, 3], [49, 72, 1844, 229, 163, 642, 3, 77, 72, 532, 163, 4, 489, 3, 767, 77, 532, 3, 169, 106, 72, 1085, 163, 37, 1052, 7, 2, 344, 4, 2204, 3, 63, 2278, 269, 87, 260, 3, 101, 163, 1557, 11, 2, 1413, 3, 1201, 3035, 3, 3241, 3, 3241, 120, 3]]


### 2.2 Calc  Encoding  and  decoding  sequence length


In [28]:
########
## Calc mean and stdev to fix  rnn_seq_in_lenght and rnn_seq_out_length
#####
X_lengths = [len(lst) for lst in X]
Y_lengths = [len(lst) for lst in Y]
#print('X_length is ', X_lengths[:2])

rnn_in_seq_mean = np.mean(X_lengths)
rnn_in_seq_stdv = np.std(X_lengths)
rnn_out_seq_mean = np.mean(Y_lengths)
rnn_out_seq_stdv = np.std(Y_lengths)

#print('Conv starter is ', conv_starter[:2])
#print('Rnn out seq is ', responses[:2])
print('rnn in seq mean & stdev :',  rnn_in_seq_mean, ' ', rnn_in_seq_stdv)
print('rnn out seq mean & stdev :',  rnn_out_seq_mean, ' ', rnn_out_seq_stdv)    

rnn in seq mean & stdev : 7.145785876993166   3.8652324774187936
rnn out seq mean & stdev : 111.35307517084283   109.5532684044888


###  2.3 Calc Rnn in/out seq lengths @ 84% coverage

In [29]:
# 1 sd away from mean = 84% coverage
rnn_in_seq_len =  rnn_in_seq_mean + rnn_in_seq_stdv
rnn_out_seq_len = rnn_out_seq_mean + rnn_out_seq_stdv

print('rnn_in_seq_len = ', rnn_in_seq_len, ' .rnn_out_seq_len :', rnn_out_seq_len)


rnn_in_seq_len =  11.01101835441196  .rnn_out_seq_len : 220.90634357533162


In [30]:
print("Total No of chats :", len(X_wordid), len(X))
trainX, trainY = X_wordid, Y_wordid

print("Sample 2 trainX :", trainX[0:2])
print("Sample 2 trainY :", trainY[0:2])


Total No of chats : 439 439
Sample 2 trainX : [[28, 72, 398, 163], [37, 49, 1844, 9, 2, 532, 4, 167, 181]]
Sample 2 trainY : [[75, 3, 8, 2900, 12, 398, 3, 37, 575, 32, 8, 2900, 3, 1201, 72, 28, 3, 1213, 18, 3, 72, 53, 3, 37, 77, 32, 1213, 439, 37, 3216, 480, 3, 1213, 2, 2411, 490, 3, 72, 53, 3, 207, 163, 72, 28, 32, 3, 10, 12, 89, 37, 132, 72, 3], [49, 72, 1844, 229, 163, 642, 3, 77, 72, 532, 163, 4, 489, 3, 767, 77, 532, 3, 169, 106, 72, 1085, 163, 37, 1052, 7, 2, 344, 4, 2204, 3, 63, 2278, 269, 87, 260, 3, 101, 163, 1557, 11, 2, 1413, 3, 1201, 3035, 3, 3241, 3, 3241, 120, 3]]


In [31]:
print('Length of x', len(trainX), len(trainY))

Length of x 439 439


### 2.4 Create Dataset : Batch major (Not Time major)

1. Because seq2seq of tf.layer accepts training data in batches
2. Encoding and decoding length fixed.

Dataset should look like


In [32]:
"""
Encode_seq : ['How', 'are' , 'you', <PAD_ID>, <PAD_ID>]
====================== Thought vectors =======================================
Decode_seq : ['<START_ID>', 'I',  'am',   'fine',   <PAD_ID>, <PAD_ID>, <PAD_ID>, <PAD_ID>  ]
Target_seq : [ 'I',         'am', 'fine', <END_ID>  <PAD_ID>, <PAD_ID>, <PAD_ID>, <PAD_ID>  ]
Target_mask: [  1           ,1     ,1      ,1        ,0          ,0         ,0      ,0      ]
"""

def getEncodeNDecode(X,Y):    
    #max_len_x = max([ len(chats) for chats in  X ]   )
    #max_len_y = max([ len(response) for response in  Y ]   )
    
    max_len_x, max_len_y = 99999999,999999999
    
    max_padlen_x =  int(rnn_in_seq_len) if (int(rnn_in_seq_len) < max_len_x ) else max_len_x
    max_padlen_y =  int(rnn_out_seq_len) if (int(rnn_out_seq_len) < max_len_y ) else max_len_y
    
    _encode_seqs = tl.prepro.pad_sequences(X, maxlen=max_padlen_x, \
                                padding='post', truncating='post', value=word2id['_pad_'] )
    
    _decode_seqs = tl.prepro.sequences_add_start_id(Y, start_id=start_id, remove_last=False)
    _decode_seqs = tl.prepro.pad_sequences(_decode_seqs, maxlen=max_padlen_y,\
                                padding='post', truncating='post', value=word2id['_pad_'] )
    
    _target_seqs = tl.prepro.sequences_add_end_id(Y, end_id=end_id)
    _target_seqs = tl.prepro.pad_sequences(_target_seqs, maxlen=max_padlen_y,\
                                padding='post', truncating='post', value=word2id['_pad_'])

    _target_mask = tl.prepro.sequences_get_mask(_target_seqs)
    
    return _encode_seqs, _decode_seqs, _target_seqs, _target_mask
    
_encode_seqs, _decode_seqs, _target_seqs, _target_mask = getEncodeNDecode(\
                                                    X_wordid[3:5], Y_wordid[3:5])

print('\n  start_id : ',word2id['start_id'],' . end_id : ', word2id['end_id'])
#print('\n_encode_seqs : ',seqId2Words( _encode_seqs) )
print('\n_encode_seqs : ',_encode_seqs) 
print('\n_decode_seqs : ',_decode_seqs)
print('\n_target_seqs : ',_target_seqs)
print('\n_target_mask : ',_target_mask)


  start_id :  3814  . end_id :  3815

_encode_seqs :  [[89, 12, 298, 248, 163, 0, 0, 0, 0, 0, 0], [29, 72, 449, 46, 7, 532, 163, 0, 0, 0, 0]]

_decode_seqs :  [[3814, 167, 248, 12, 3657, 3, 32, 163, 2, 3657, 3, 75, 3, 107, 3657, 3, 10, 12, 167, 248, 252, 3, 411, 163, 72, 297, 26, 2689, 3, 32, 411, 3, 89, 12, 298, 248, 163, 3657, 3, 10, 12, 167, 248, 252, 3, 3657, 3, 1201, 163, 72, 104, 88, 163, 37, 3216, 269, 89, 72, 869, 3, 1299, 1201, 72, 79, 163, 167, 769, 3, 72, 28, 2351, 229, 3, 10, 12, 185, 3, 75, 18, 3598, 3, 112, 89, 12, 185, 163, 89, 12, 298, 1207, 163, 167, 1207, 163, 37, 3216, 269, 37, 575, 80, 2238, 10, 12, 59, 205, 3, 77, 72, 754, 229, 439, 163, 1201, 163, 89, 79, 72, 241, 5, 269, 163, 37, 49, 104, 5, 269, 684, 3, 37, 77, 32, 754, 72, 684, 3, 124, 107, 345, 6, 687, 48, 39, 145, 115, 18, 3, 37, 3216, 269, 98, 5, 345, 3, 20, 2, 641, 3, 89, 12, 2, 641, 163, 89, 834, 44, 2, 641, 163, 2, 467, 3, 2, 2186, 163, 75, 3, 1201, 72, 28, 3, 89, 575, 37, 163, 8, 2900, 3, 483, 79, 72, 2

In [33]:
def seqId2Words(seq, get_first_n_words_only = 5):
    lst = []
    for word_lst in seq:
        #print(word_lst)
        lst.append([ id2word[word_id] if word_id in id2word.keys() else word_id\
                    for word_id  in word_lst[0:get_first_n_words_only]  ] )            
    return  lst

lst = [[104, 16, 394, 313, 0, 0, 0, 0, 0], [35, 83, 663, 53, 8, 837, 0, 0, 0]]

seqId2Words(lst)

[['like', 'he', 'strong', 'free', '_pad_'],
 ['they', 'could', 'compared', 'first', 'a']]

In [34]:
id2word[104]
word2id['.']
#gl

3

## 3.0 Rnn Modeling

### 3.1 Rnn param

In [65]:
learning_rate = 0.0001
batch_size = 4
xvocab_size = len(id2word) 
emb_dim = emb_dims
rnn_num_layers = 1
rnn_num_hidden_nodes = 32
dropout_rate = 0.2


#  decay learning rate by 25% every 10 step
lr_decay_rate =  0.75
lr_decay_steps = 10
max_grad_clip_to = 5.0

### 3.1.1 Word Embedding Dict to Array

Because : EmbeddingInput layers expects the word embeddings to be in array format

In [36]:
def getWordEmbeddingMatrix(word2id, wordvec_embedding):
    word_embedding_matrix = []
    for  word in word2id:    
        #print('word not found :', word)
        if word not in ('start_id','end_id'):
            word_embedding_matrix.append(list(wordvec_embedding[word]) )              
    return word_embedding_matrix        

word_embedding_matrix = getWordEmbeddingMatrix(word2id, wordvec_embedding)
print( word_embedding_matrix[0:2])

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.9556146577706295, -0.13992412012575106, -0.056778121857022246, -0.05578928587387999, -0.3326397804950836, -0.5815672313892881, -0.6897877349801806, -0.4392319148462587, 0.9596660690177858, 0.23127292264927224, -0.6544596923728019, -0.12045775254096824, 0.2859743500718541, 0.4461833420822854, -0.9079563483546558, 0.7783693189280061, -0.07089576084196958, -0.6940928407847575, 0.21880936735927614, -0.2122010949993265, -0.7695859257099906, -0.4434590950489752, -0.9335484021055345, 0.8009717392555136, -0.3273243569590827, 0.47691780200103917, 0.943103382941558, -0.7072788852353091, 0.8702251345225276, 0.8974103106471725, -0.12553697499315497, 0.44361548472735524, 0.660990804311048, 0.8158677075553986, -0.329341895832826, -0.577594933017182

In [41]:
word_embedding_matrix[1][0:3] 

[0.9556146577706295, -0.13992412012575106, -0.056778121857022246]

### Word Embedding Matrix : Inference / Prediction Purpose

Creating full embedding matrix from embedding dictionary to be used in the decoding layer

In [38]:
all_word_embedding_matrix = getWordEmbeddingMatrix(all_word2id, all_wordvec_embedding)

print('length of word_embedding_matrix = ', len(word_embedding_matrix),\
       '. all_word_embedding_matrix', len(all_word_embedding_matrix))
assert len(word_embedding_matrix) < len(all_word_embedding_matrix) /10 , "Train Embed matrix must be smaller than Inference."

length of word_embedding_matrix =  3814 . all_word_embedding_matrix 400159


## 3.3 Tf Placeholder

1. **Training Input Placeholder** :
2. **Inference Input Placeholders**

In [70]:
tf.reset_default_graph()

# model for training
encode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="encode_seqs")
decode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="decode_seqs")
target_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_seqs")
target_mask = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_mask") 
# tl.prepro.sequences_get_mask()


###  3.2 RNN Many to Many Model Defn

1. **Encoding layer**: Look up layer to lookup word id into embedding.
2. **Decoding layer**: Look up layer to lookup word id into embedding.
3. **Seq2Seq layer**: A simple dynamic Rnn layer.
      

REF: https://tensorlayer.readthedocs.io/en/stable/modules/layers.html#tensorlayer.layers.Seq2Seq

In [55]:
print( len( word_embedding_matrix), len( word_embedding_matrix[0]) )
print( len( all_word_embedding_matrix), len( all_word_embedding_matrix[0]) )
rnn_in_seq_len
encode_seq_len = int(rnn_in_seq_len)
encode_seq_len


3814 50
400159 50


11

In [96]:

tf.reset_default_graph()

# model for training
encode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="encode_seqs")
decode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="decode_seqs")
target_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_seqs")
target_mask = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_mask") 
# tl.prepro.sequences_get_mask()

##################################

def model(encode_seqs, decode_seqs, is_train=True, reuse=False):
    with tf.variable_scope("model", reuse=reuse):
        encode_seq_len = int(rnn_in_seq_len)
        decode_seq_len = int(rnn_out_seq_len)


        embed_words, embed_word_dim = len( word_embedding_matrix) , len( word_embedding_matrix[0])
        encoder_emb_layer = tf.get_variable(name="encoder_emb_layer", shape=[embed_words, embed_word_dim],\
                        initializer=tf.constant_initializer(word_embedding_matrix), trainable=False)

        ## Convert encode_seqs to  embedded_encode_seqs
        ## [batch_size, seq_len] ===>>  [batch_size, seq_len, feat]
        # STEPS: [batch_size, seq_len] ===>> [batch_size*seq_len,1] => embed_lookup => # [batch_size, seq_len, feat]
        encoder_emb_inp = tf.reshape(encode_seqs, [batch_size*encode_seq_len,1])
        encoder_emb_inp = tf.nn.embedding_lookup(encoder_emb_layer, encoder_emb_inp)
        encoder_emb_inp = tf.reshape(encoder_emb_inp, [batch_size, encode_seq_len, emb_dim])


        all_embed_words, all_embed_word_dim = len( all_word_embedding_matrix) , len( all_word_embedding_matrix[0])
        decoder_emb_layer = tf.get_variable(name="decoder_emb_layer", shape=[all_embed_words, all_embed_word_dim],\
                        initializer=tf.constant_initializer(all_word_embedding_matrix), trainable=False)

        ## Repeat same process for decode seq
        ## [batch_size, seq_len] ===>>  [batch_size, seq_len, feat]
        # STEPS: [batch_size, seq_len] ===>> [batch_size*seq_len,1] => embed_lookup => # [batch_size, seq_len, feat]
        decoder_emb_inp = tf.reshape(decode_seqs, [batch_size*decode_seq_len,1])
        decoder_emb_inp = tf.nn.embedding_lookup(decoder_emb_layer, decoder_emb_inp)
        decoder_emb_inp = tf.reshape(decoder_emb_inp, [batch_size, decode_seq_len, emb_dim])

        #####################
        ## Define Encoder
        ####################
        #encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=rnn_num_hidden_nodes)
        encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=rnn_num_hidden_nodes)
        initial_state = encoder_cell.zero_state(batch_size, dtype=tf.float32)
        encoder_outputs, encoder_state = tf.nn.dynamic_rnn(
            encoder_cell, encoder_emb_inp, initial_state=initial_state,    
            #sequence_length=[batch_size for _ in range(encode_seq_len)], 
            #sequence_length=[encode_seq_len for _ in range(batch_size)], 
            time_major=False, swap_memory=True)

        #####################
        ## Define Decoder
        ####################
        from tensorflow.python.layers.core import Dense
        # Build RNN cell
        decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units= rnn_num_hidden_nodes)
        projection_layer = Dense(units=xvocab_size, use_bias=True, activation=tf.nn.relu)
        # A Helper to be used during training only.
        helper = tf.contrib.seq2seq.TrainingHelper(  decoder_emb_inp,\
                        [ decode_seq_len for _ in range(batch_size)], time_major=False           )

        decoder = tf.contrib.seq2seq.BasicDecoder(
                decoder_cell, helper, encoder_state,  output_layer=projection_layer)
        # Dynamic decoding
        net_out, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder, output_time_major=False,  swap_memory=True  )

    return net_out, _



### 3.3.1 Model Train Defn

VIMP : During training, the model should not be reused because we will be iteratively making it more and more better. However in test  case, we must set the model reuse to true, as we do want the previously trained model without any modification.

In [97]:
net_out, _ = model(encode_seqs, decode_seqs, is_train=True, reuse=False)

In [112]:
# model for inferencing
encode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="encode_seqs")
decode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="decode_seqs")
net, net_rnn = model(encode_seqs2, decode_seqs2, is_train=False, reuse=True)
#y = tf.nn.softmax(net.rnn_output)
net.rnn_output

<tf.Tensor 'model_4/decoder/transpose:0' shape=(4, ?, 3816) dtype=float32>

In [ ]:
### TO DO : Compute cosine similarity of y with all_embedding_matrix to retrieve  predicted words

## 3.4 Loss  Defn

1. **Mask Loss**: is used. Mask is multiplied by calculated loss, to ensure padding losses are not accounted during loss calculation

In [102]:
net_out.rnn_output

<tf.Tensor 'model/decoder/transpose:0' shape=(4, ?, 3816) dtype=float32>

In [105]:
#n_classes = xvocab_size
#logits =tf.reshape(net_out.rnn_output, [batch_size*decode_seq_len,n_classes])


In [107]:
##logits : expected shape 2d [batch_size*:,n_classes]
n_classes = xvocab_size
logits =tf.reshape(net_out.rnn_output, [batch_size*decode_seq_len,n_classes])

#  we calculate  cross entropy with mask 
# because we do not want  the padding to have any effect on the lonss
loss = tl.cost.cross_entropy_seq_with_mask(logits=logits, target_seqs=target_seqs,\
                                    input_mask=target_mask, return_details=False, name='cost')

### 3.4.1 Optimizer Defn : Decaying rate & Grad Clip

1. **Vanishing Gradient :** Since we are using relu, vanishing gradient is not a  big problem. 
2. **Exploding Gradient :** : Clipping gradients to prevent the problem
3. **Optimiser :** Adam Optimisers tend to converge faster. SGD although slow tend to outperfom Adam however
4. **Decaying learning rate**: For adam it is not required, because ADAM guarantees square root decay as per theorem 4.1 

t <- t +1

lr_t <- learning_rate * sqrt(1 - beta2^t) / (1 - beta1^t)

PS:  Theorem 4.1 of their ICLR article, one of their hypotheses is that the learning rate has a square root decay, αt=α/t√. Furthermore, for their logistic regression experiments they use the square root decay as well. 
Ref: https://stats.stackexchange.com/questions/200063/adam-optimizer-with-exponential-decay

In [108]:
global_step = tf.Variable(0, trainable=False)
inc_gstep = tf.assign(global_step,global_step + 1)

# Learning rate decay ~ lr * decay_rate ^(global_step/ decay_step)
print('learning rate :', learning_rate)
#learning_rate = tf.train.exponential_decay(
#    lr, global_step, decay_steps=lr_decay_steps,\
#    decay_rate=lr_decay_rate, staircase=True)

#lr = tf.placeholder(tf.float32, name = 'learning_rate')
# We define Adam Optimizer
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Gradient clipping
#gradients, v = zip(*optimizer.compute_gradients(loss))
#gradients, _ = tf.clip_by_global_norm(gradients, max_grad_clip_to)
#optimizer = optimizer.apply_gradients(
#    zip(gradients, v))

#train_op = optimizer

#net_out.print_params(False)
#train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)


learning rate : 0.0001


# 4.0 Model Training

In [45]:
len(trainX)
len(trainY)

439

In [46]:
trainX

[[28, 72, 398, 163],
 [37, 49, 1844, 9, 2, 532, 4, 167, 181],
 [34, 12, 2, 2044, 163],
 [89, 12, 298, 248, 163],
 [29, 72, 449, 46, 7, 532, 163],
 [77, 72, 2968, 439, 163],
 [77, 72, 2265, 229, 55, 163],
 [29, 72, 1380, 5, 442, 163],
 [37, 218, 10, 72, 28, 8, 586, 284, 457, 5, 1544, 229, 3],
 [1600, 3, 107, 3216, 3030, 229, 3],
 [169, 149, 29, 48, 46, 2758, 163],
 [89, 212, 18, 869, 5, 26, 348, 163],
 [48, 164, 638, 54, 532, 69, 51, 3],
 [169, 157, 79, 72, 2081, 163],
 [169, 157, 2008, 79, 72, 29, 163],
 [28, 72, 8, 2111, 163],
 [89, 12, 298, 248, 163],
 [79, 72, 104, 240, 163],
 [30, 1042, 3],
 [79, 72, 532, 229, 163, 252, 163],
 [37, 575, 1805, 3],
 [30, 37, 241, 69, 1497, 3],
 [37, 575, 681, 5, 2907, 3],
 [37, 83, 26, 298, 769, 3],
 [12, 18, 8, 1512, 163],
 [79, 72, 269, 163],
 [79, 72, 104, 649, 54, 229, 163],
 [1497, 3],
 [37, 107, 3216, 921, 72, 388, 3],
 [37, 80, 568, 72, 1550, 3],
 [181, 3598, 905, 3],
 [72, 28, 107, 25, 2776, 3],
 [167, 586, 12, 986, 3],
 [124, 163, 39, 63, 21

In [109]:
#learning_rate = 0.3

import time
from sklearn.utils import shuffle

sess = tf.Session()
sess.run( tf.global_variables_initializer() )

n_epoch = 1
for epoch in range(n_epoch):
    
    total_err, steps = 0, 0
    epoch_time = time.time()    
    trainX, trainY = shuffle(trainX, trainY, random_state=0)
    
    # use minibatches generated from trainX, trainY 
    for X, Y in tl.iterate.minibatches(inputs=trainX, targets=trainY,\
                                batch_size=batch_size, shuffle=False):
        step_time = time.time()
        #_encode_seqs, _decode_seqs, _target_seqs, _target_mask = getEncodeNDecode(X,Y)
        
        #print(_encode_seqs)

In [52]:
#all_wordvec_embedding['birthday']

In [110]:
import time
from sklearn.utils import shuffle

sess = tf.Session()
sess.run( tf.global_variables_initializer() )

n_epoch = 200000
bot_used_words = []

for epoch in range(n_epoch):    
    total_err, steps = 0, 0
    epoch_time = time.time()    
    trainX, trainY = shuffle(trainX, trainY, random_state=0)
    
    
    # use minibatches generated from trainX, trainY 
    for X, Y in tl.iterate.minibatches(inputs=trainX, targets=trainY,\
                                batch_size=batch_size, shuffle=False):
        step_time = time.time()
        _encode_seqs, _decode_seqs, _target_seqs, _target_mask = getEncodeNDecode(X,Y)
        
        _, err = sess.run([train_op, loss],
                        {encode_seqs: _encode_seqs,    decode_seqs: _decode_seqs,
                        target_seqs: _target_seqs,     target_mask: _target_mask } )
        #                lr: learning_rate }  )
        total_err += err; 
        
        #if steps % (100) == 0:
        if epoch % (2000) == 0 and steps   == 0:
            print("Epoch[%d/%d] Iter: %d loss:%f learning rate:%f " \
                    % (epoch, n_epoch, steps,  err, 1) )
            #print('global steps is ', global_step)
        steps += 1

                    
    ##########
    ## Decay Learning rate @ every epoch
    ##########
    #learning_rate = learning_rate * lr_decay_rate
    #if learning_rate < min_learning_rate:
    #    learning_rate = min_learning_rate
        
        
    print("Epoch[%d/%d] averaged loss:%f took:%.5fs Bot Words %d / %d" \
          % (epoch, n_epoch, total_err/steps, time.time()-epoch_time, len(bot_used_words) , len(word2id) ))
    

print('Completed')

Epoch[0/200000] Iter: 0 loss:8.253335 learning rate:1.000000 
Epoch[0/200000] averaged loss:8.201649 took:20.35158s Bot Words 0 / 3816
Epoch[1/200000] averaged loss:8.028055 took:18.77680s Bot Words 0 / 3816
Epoch[2/200000] averaged loss:7.777000 took:18.82467s Bot Words 0 / 3816
Epoch[3/200000] averaged loss:7.519334 took:19.76317s Bot Words 0 / 3816
Epoch[4/200000] averaged loss:7.292147 took:18.86457s Bot Words 0 / 3816
Epoch[5/200000] averaged loss:7.091491 took:18.83564s Bot Words 0 / 3816
Epoch[6/200000] averaged loss:6.911118 took:18.13452s Bot Words 0 / 3816
Epoch[7/200000] averaged loss:6.746180 took:18.18638s Bot Words 0 / 3816
Epoch[8/200000] averaged loss:6.606684 took:18.44269s Bot Words 0 / 3816
Epoch[9/200000] averaged loss:6.495537 took:20.35658s Bot Words 0 / 3816
Epoch[10/200000] averaged loss:6.385993 took:18.99023s Bot Words 0 / 3816
Epoch[11/200000] averaged loss:6.280138 took:18.34795s Bot Words 0 / 3816
Epoch[12/200000] averaged loss:6.193547 took:18.83166s Bot W

KeyboardInterrupt: 

## V2.0 : Faster Training + Infer words not in training
1. Using small training embeddings
2. For inference  using large / full word vec embeddings

### V 3.0 Must not remove .
Because if we do not train it with end of sentence, then it is notgoing to be able to learn, when to enter full stop.

# 4.0 Computation Space/ time reduction :

1. Negative Sampling for target sequences :
2. Training : Smaller / limited WordVector embeddings
3. Prediction : Trained embedding + Bigger embedding containing words not in the trained embeddings

During training, use  only the words in the training instance. But since we want to be able to predict words that  are limited not only  by the  training words. During the prediction or chat bot resonse generation phase, we can use bigger embedding matrix for lookup to supplement the trained embedding.


## 5.0 Future  :  To Dos



1. **Loss Function** : Evaluate it against other loss functions such as perplexity, BLEU or other measures
2. **Beam Search**: Increased performance implied by literatures.
3. **Attention Mechanism** : Increased performance implied by literatures.
